<a href="https://colab.research.google.com/github/xBosse/notebooks/blob/main/JsonFormatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json, random
!git clone https://github.com/xBosse/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 232 (delta 1), reused 0 (delta 0), pack-reused 226
Receiving objects: 100% (232/232), 1.38 MiB | 4.04 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [ ]:
ls

datasets/  sample_data/


# JSON als Txt einscannen und umformatieren

**METHODEN**

In [ ]:


import numpy as np


def indexToString_v5(pos_list, max):     # BIO Schema, mehrere SubjExpr pro Einheit
  array = np.zeros(max, dtype=int).tolist()
  for pos in pos_list:  
    i = 0
    for x in pos:
      array[int(x)-1] = 1 + i
      i = 1

  array_str = str(array).replace(" ", "")
  return "subj\": " + array_str + "}"

def indexToString_v6(pos_list, max):     # copy
  array = np.zeros(max, dtype=int).tolist()
  for pos in pos_list:  
    i = 0
    for x in pos:
      array[int(x)] = 1 + i
      i = 1

  array_str = str(array).replace(" ", "")
  return "subj\": " + array_str + "}"

  ##############################
def JsonToString_v2(lines):
  newlines = []
  last_line = ""
  new_line = ""
  pos_list = []
  pos = []

  ######## erster satz ###########

  # Wöter Liste zuschneiden und Namen austauschen
  # words
  last_line = lines[0]
  new_line = lines[0][:lines[0].find("LBL")]
  max = new_line.count(", ")
  new_line = new_line.replace("seq_words", "words")

  # Position of Subj finden und als pos[] in pos_list
  # subj
  a = lines[0].find("pred_sense")   
  b = lines[0][a + 15:]
  c = b.find("]")   
  d = b[:c]
  d = d.strip()
  pos = d.split(",")
  pos_list.append(pos)
  pos.clear
  lines.pop(0)

  for line in lines:


    if(last_line[:last_line.find("LBL")] == line[:line.find("LBL")] ): # Falls gleicher Satz, keine neue Zeile, nur den subjektiven Ausdruck der letzten Zeile hinzufügen 

      # subj
      a = line.find("pred_sense")   
      b = line[a + 15:]
      c = b.find("]")   
      d = b[:c]
      d = d.strip()
      pos = d.split(",")
      pos_list.append(pos)
      pos.clear

    else:  # neuer Satz  -> neue Zeile
      # vorherigen Satz fertigstellen      
      newlines.append(new_line + indexToString_v5(pos_list, max))
      pos_list.clear()

      last_line = line     
      # words
      new_line = line[:line.find("LBL")]
      max = new_line.count(", ")
      new_line = new_line.replace("seq_words", "words")     
      # subj
      a = line.find("pred_sense")   
      b = line[a + 15:]
      c = b.find("]")   
      d = b[:c]
      d = d.strip()
      pos = d.split(",")
      pos_list.append(pos)
      pos.clear
  ###### letzten satz fertigstellen ######
  newlines.append(new_line + indexToString_v5(pos_list, max))

  return newlines

##############################
def list_to_string(list):
  string = ""
  for x in list:
    string = string + str(x) + "\n"
  return string

###########################
def train_val_splits(lines):  # Teilt den Datensatz in Trainings- Validationssplits auf und speichert diese
  random.shuffle(lines)

  cutoff = int(len(lines) * 0.9)
  lines_train = lines[:cutoff]
  lines_val = lines[cutoff:]

  print("Train: " + str(len(lines_train)) + " Val: " + str(len(lines_val)))


  json_string = list_to_string(lines_train)  
  text_file = open("train_shata14.txt", "w")
  text_file.write(json_string)
  text_file.close()

  json_string = list_to_string(lines_val)    
  text_file = open("val_shata14.txt", "w")
  text_file.write(json_string)
  text_file.close()



Einlesen + Umformatieren

In [ ]:
##
##  shata14
##

# Reading as Txt file
newlines = []
with open('datasets/shata14_v5.jsonl') as f:
    lines = f.readlines()
# Closing file
f.close()

newlines = JsonToString_v2(lines)  # Formatiert das Json-Format um in die zwei Listen: words und subj

json_string = list_to_string(newlines) 

#write to file
text_file = open("shata14.txt", "w")
text_file.write(json_string)
text_file.close()

#TRAIN and VALIDATION SPLITS

train_val_splits(newlines)



##
##  STEPS16
##

# Reading as Txt file
newlines = []
with open('datasets/steps16_v1.jsonl') as f:
    lines = f.readlines()
# Closing file
f.close()

newlines = JsonToString_v2(lines)

json_string = list_to_string(newlines)

#write to file
text_file = open("steps16.txt", "w")
text_file.write(json_string)
text_file.close()

#TEST SPLIT
text_file = open("test_steps16.txt", "w")
text_file.write(json_string)
text_file.close()

Train: 520 Val: 58


#ALTER CODE ---- in Datenerweiterungen übernommen - Für Scoring: für jedes subjExpr

jedes SubjExpr den Index dessen Satzes anhängen

In [ ]:
# Reading as Txt file

with open('datasets/steps16_v1.jsonl') as f:
    lines = f.readlines()
# Closing file
f.close()



In [ ]:
import json

string = "{ \"Steps16\": ["

for x in lines:
  string += x + ","

string = string[:-1]

string += "]}"



data = json.loads(string)

print(data)



{'Steps16': [{'seq_words': ['Die', 'Grünen', 'empfehlen', 'Ihnen', ',', 'dieses', 'Gebäudeprogramm', 'zu', 'unterstützen', ',', 'dies', 'sowohl', 'aus', 'wirtschaftlichen', 'als', 'auch', 'aus', 'ökologischen', 'Gründen', '.'], 'LBL': ['B-Source', 'I-Source', 'O', 'B-Target', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'pred_sense': [[3], ['empfehlen'], ['VVFIN']], 'src_lang': '<DE>', 'tgt_lang': '<DE-ORL>'}, {'seq_words': ['Die', 'Grünen', 'empfehlen', 'Ihnen', ',', 'dieses', 'Gebäudeprogramm', 'zu', 'unterstützen', ',', 'dies', 'sowohl', 'aus', 'wirtschaftlichen', 'als', 'auch', 'aus', 'ökologischen', 'Gründen', '.'], 'LBL': ['O', 'O', 'O', 'B-Source', 'O', 'B-Target', 'I-Target', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'pred_sense': [[9], ['unterstützen'], ['VVINF']], 'src_lang': '<DE>', 'tgt_lang': '<DE-ORL>'}, {'seq_words': ['Die', 'Grünen', 'empfehlen', 'Ihnen', ',', 'dieses', 'Gebäudeprogramm', 'zu', 'unterstützen', 

In [ ]:
# scor_set = [ labels = []]
scorer_set = []

i = -1
last_words = data["Steps16"][0]["seq_words"]
scorer_line = []
c = 0
for line in data["Steps16"]:
  c +=1



  if line["seq_words"] == last_words:
    #scorer_line.append(i)
    scorer_line.append(line["pred_sense"][0])
  else:
    last_words = line["seq_words"]
    i += 1
    scorer_set.append(scorer_line)
    scorer_line = []

    #scorer_line.append(i)
    scorer_line.append(line["pred_sense"][0])



scorer_set.append(scorer_line)



In [ ]:
for x in scorer_set:
  print(x)

[[3], [9], [3], [7]]
[[8], [13], [16, 17, 18], [24, 27], [30]]
[[4], [15], [29], [31], [31]]
[[2, 4], [17]]
[[2], [11], [13], [16, 17]]
[[18], [26], [21, 27], [28], [18], [18]]
[[13], [21], [29], [42], [43], [21]]
[[4], [7], [14], [15, 16, 17]]
[[2], [3], [12, 14], [18], [19]]
[[11], [29, 30], [36], [39], [43], [48], [48], [39]]
[[2], [3], [9], [9]]
[[4], [6], [11], [13], [14], [19], [20], [4]]
[[2, 4], [29], [17]]
[[2], [7]]
[[9], [9]]
[[2], [4], [7], [21], [20]]
[[13], [18], [12]]
[[4], [9], [14], [19], [27], [26], [30], [35], [41], [44], [30], [27]]
[[3], [10], [18], [9], [37], [14, 15]]
[[2], [11], [8]]
[[6], [15], [23], [20]]
[[7], [16], [17, 18, 19], [28], [26], [34], [36], [40], [42]]
[[3, 11], [16], [19], [3, 11]]
[[2], [4, 5], [10], [14], [18], [23], [24, 25]]
[[2], [9], [10, 11, 12], [20], [25], [31], [32], [34], [39], [26]]
[[2], [10], [7], [7], [2]]
[[7], [8], [13, 17], [13, 17], [13, 17]]
[[4], [2], [5], [13], [22], [26, 30], [13]]
[[9], [11], [13], [22], [23], [28], [32],

# ALTER CODE --------- Creating/Splitting into Train Validation Test splits 

In [ ]:
# Reading as Txt file
newlines = []
with open('datasets/shata_steps.txt') as f:
    lines = f.readlines()
    
    count = 0
    for x in lines:
      #print(x)         
      count = count + 1
      if count > 5:
        break

# Closing file
f.close()

l = len(lines)
train = l * 0.8
val = l * 0.1
test = l * 0.1

print("Train: " + str(train) + " Val: " + str(val) + " Test: " + str(test))

import random
random.shuffle(lines)

len(lines)
lines_train = lines[:3716]
lines_val = lines[3716:-464]
lines_test = lines[-464:]

print("Train: " + str(len(lines_train)) + " Val: " + str(len(lines_val))  + " Test: " + str(len(lines_test)) )



json_string = list_to_string(lines_train)

text_file = open("train.txt", "w")
text_file.write(json_string)
text_file.close()

json_string = list_to_string(lines_val)

text_file = open("val.txt", "w")
text_file.write(json_string)
text_file.close()

json_string = list_to_string(lines_test)

text_file = open("test.txt", "w")
text_file.write(json_string)
text_file.close()

4645

# ALTER CODE ------------- In oberen teil übernommen Train Val Split

In [ ]:
!git clone https://github.com/xBosse/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 145 (delta 71), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (145/145), 701.12 KiB | 1.62 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [ ]:
# Reading as Txt file
newlines = []
with open('datasets/steps16.txt') as f:
    lines = f.readlines()
    
    count = 0
    for x in lines:
      #print(x)         
      count = count + 1
      if count > 5:
        break

# Closing file
f.close()

In [ ]:
l = len(lines)
train = l * 0.9
val = l * 0.1


print("Train: " + str(train) + " Val: " + str(val) )

Train: 2175.3 Val: 241.70000000000002


In [ ]:
import random
random.shuffle(lines)

len(lines)
lines_train = lines[:2175]
lines_val = lines[2175:]


print("Train: " + str(len(lines_train)) + " Val: " + str(len(lines_val)))


json_string = ''.join(lines_train)

text_file = open("train_steps16.txt", "w")
text_file.write(json_string)
text_file.close()

json_string = ''.join(lines_val)

text_file = open("val_steps16.txt", "w")
text_file.write(json_string)
text_file.close()

Train: 2175 Val: 53


#ALTER CODE -------- Zeilen kombinieren

In [ ]:




import numpy as np


def indexToString_v555(pos_list, max):     # BIO Schema, mehrere SubjExpr pro Einheit
  array = np.zeros(max, dtype=int).tolist()
  for pos in pos_list:  
    i = 0
    for x in pos:
      array[int(x)] = 1 + i
      i = 1

  array_str = str(array).replace(" ", "")

  return "subj\": " + array_str




  ##############################
def JsonToString_v222(lines):
  newlines = []

  last_line = ""
  new_line = ""

  pos_list = []
  pos = []

  ######## erster satz ###########

  # Wöter Liste zuschneiden und Namen austauschen
  # words
  last_line = lines[0]
  new_line = lines[0][:lines[0].find("LBL")]
  max = new_line.count(", ")
  new_line = new_line.replace("seq_words", "words")

  # Position of Subj finden und als pos[] in pos_list
  # subj
  a = lines[0].find("pred_sense")   
  b = lines[0][a + 15:]
  c = b.find("]")   
  d = b[:c]
  d = d.strip()
  pos = d.split(",")
  pos_list.append(pos)
  pos.clear

  lines.pop(0)

  for line in lines:
    print("l1ne: " + line)
    print("l2ne: " + last_line)

    if(last_line[:last_line.find("LBL")] == line[:line.find("LBL")] ): # gleicher Satz

      # subj
      a = line.find("pred_sense")   
      b = line[a + 15:]
      c = b.find("]")   
      d = b[:c]
      d = d.strip()
      pos = d.split(",")
      pos_list.append(pos)
      pos.clear
      

    else:  # neuer Satz   

      # vorherigen Satz fertigstellen
      
      print("adding line")

      newlines.append(new_line + indexToString_v5(pos_list, max))
      pos_list.clear()

      last_line = line
      
      # words
      new_line = line[:line.find("LBL")]
      max = new_line.count(", ")
      new_line = new_line.replace("seq_words", "words")
      
      # subj
      a = line.find("pred_sense")   
      b = line[a + 15:]
      c = b.find("]")   
      d = b[:c]
      d = d.strip()
      pos = d.split(",")
      pos_list.append(pos)
      pos.clear

  ########### letzter satz ##############
  print("last line")
  newlines.append(new_line + indexToString_v5(pos_list, max))

  return newlines


In [ ]:
newlines = []
with open('datasets/shata_test.jsonl') as f:
    lines = f.readlines()
# Closing file
f.close()



In [ ]:
  ########## Testlauf #############


##
##  SHATA14 TEST
##

# Reading as Txt file
newlines = []
with open('datasets/steps16_v1.jsonl') as f:
    lines = f.readlines()
# Closing file
f.close()

newlines = JsonToString_v3(lines)

json_string = list_to_string(newlines)

#write to file
text_file = open("steps16_test.txt", "w")
text_file.write(json_string)
text_file.close()

#TRAIN and VALIDATION SPLITS

train_val_splits(newlines)


#ALTER CODE ---------- Alte Methoden

In [ ]:
def indexToString_v1(pos, max):      # tags = 0,1

  zero = ""

  for a in range(max - 1):
    zero += "0,"
  
  zero = zero + "0]}"

  for x in pos:
    i = (int(x) -1) * 2
    zero = zero[:i] + "1" + zero[i+1:]

  return "subj\": [" + zero




def indexToString_v2(pos, max):     # tags = S,O

  zero = ""

  for a in range(max - 1):
    zero += "\"O\","
  
  zero = zero + "\"O\"]}"

  for x in pos:
    i = (int(x) -1) * 4 + 1
    zero = zero[:i] + "S" + zero[i+1:]


  return "subj\": [" + zero



def indexToString_v3(pos, max):     # BIO tags = B,I,O string

  zero = ""

  for a in range(max - 1):
    zero += "\"O\","
  
  zero = zero + "\"O\"]}"

  j = 0
  for x in pos:
    i = (int(x) -1) * 4 + 1

    if(j==0):
      zero = zero[:i] + "B" + zero[i+1:]
    else:
      zero = zero[:i] + "I" + zero[i+1:]
    j = j+1

  return "subj\": [" + zero

##########################     
def indexToString_v4(pos, max):     # BIO  tags = 0,1,2  String
  zero = ""
  for a in range(max - 1):
    zero += "0,"
  
  zero = zero + "0]}"

  j = 0
  for x in pos:
    i = (int(x) -1) * 2

    if(j==0):
      zero = zero[:i] + "1" + zero[i+1:]
    else:
      zero = zero[:i] + "2" + zero[i+1:]
    j = j+1

  return "subj\": [" + zero

##############################
def JsonToString(lines):
  newlines = []
  for line in lines:
    newwords = ""
    # words
    a = line.find("LBL")
    newwords += line[:a]
    max = newwords.count(", ")
    newwords = newwords.replace("seq_words", "words")

    # subjExp TAG
    a = line.find("pred_sense")   
    b = line[a + 15:]
    c = b.find("]")   
    d = b[:c]
    d = d.strip()
    pos = d.split(",")

    newwords += indexToString_v4(pos, max) ########### Hier Tag-Array-formatting-function

    newlines.append(newwords)
  return newlines


